In [2]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import yfinance as yf

C:\python\Lib\site-packages\yfinance\base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [3]:
#implementation of BSM model
def bsm_model(S, K, T, r, sigma, option_type='call'):
    d1= (np.log(S/K) + ( r + (sigma**2)/2)*T)/sigma*np.sqrt(T)
    d2=d1-sigma*np.sqrt(T)
    if option_type == 'call':
        option_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        option_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    return option_price

In [4]:
symbol = "AAPL"

def process_expiration(exp_td_str):
    options = tk.option_chain(exp_td_str)
    calls = options.calls
    calls['expiration_date'] = exp_td_str  
    return calls

tk = yf.Ticker(symbol)  #tk is the object 

# Get the available expiration dates for options
expirations = tk.options
# print(expirations) #will give list of all available expiration dates
# type(expirations) #it will give me tuple
# Create an empty DataFrame to store option data
data = pd.DataFrame()

# Loop through each expiration date and process option data
for exp_td_str in expirations:
    exp_data = process_expiration(exp_td_str)
    data = pd.concat([data, exp_data], ignore_index=True)

# Convert expiration date string to datetime
# print(data['expiration_date'])
data['expiration_date'] = pd.to_datetime(data['expiration_date'])
# print(data['expiration_date'])
# Calculate time to expiration in years
data['T'] = (data['expiration_date'] - pd.Timestamp.today()).dt.days / 365
data['T']=data['T'].values
print((data['T']))
def get_closing_price(ticker):
    stock_data = yf.download(ticker, period='1d')
    return stock_data['Close'].iloc[-1]

aapl_closing_price = get_closing_price('AAPL')
r = 0.0425 #risk free interest rate

# Calculate option prices using BSM model
data['bsm_price'] = bsm_model(aapl_closing_price, data['strike'], data['T'], r, data['impliedVolatility'], option_type='call')


# Display the DataFrame with BSM prices
df=pd.DataFrame(data[['contractSymbol', 'expiration_date', 'strike', 'lastPrice', 'bsm_price']])
df

0      0.008219
1      0.008219
2      0.008219
3      0.008219
4      0.008219
         ...   
939    2.578082
940    2.578082
941    2.578082
942    2.578082
943    2.578082
Name: T, Length: 944, dtype: float64


[*********************100%%**********************]  1 of 1 completed


,contractSymbol,expiration_date,strike,lastPrice,bsm_price
0,AAPL240524C00100000,2024-05-24,100.0,83.60,8.990492e+01
1,AAPL240524C00105000,2024-05-24,105.0,82.30,8.490667e+01
2,AAPL240524C00115000,2024-05-24,115.0,59.90,7.491016e+01
3,AAPL240524C00120000,2024-05-24,120.0,57.80,6.991191e+01
4,AAPL240524C00125000,2024-05-24,125.0,61.20,6.491365e+01
...,...,...,...,...,...
939,AAPL261218C00310000,2026-12-18,310.0,5.60,2.143784e-20
940,AAPL261218C00320000,2026-12-18,320.0,4.45,5.748160e-24
941,AAPL261218C00330000,2026-12-18,330.0,3.78,1.052686e-27
942,AAPL261218C00340000,2026-12-18,340.0,3.28,1.372355e-31


In [7]:
df[df['strike']==160]

,contractSymbol,expiration_date,strike,lastPrice,bsm_price
13,AAPL240524C00160000,2024-05-24,160.0,30.05,29.925876
54,AAPL240531C00160000,2024-05-31,160.0,29.00,30.056188
96,AAPL240607C00160000,2024-06-07,160.0,30.85,30.186394
125,AAPL240614C00160000,2024-06-14,160.0,31.27,30.316494
176,AAPL240621C00160000,2024-06-21,160.0,30.70,30.446488
207,AAPL240628C00160000,2024-06-28,160.0,31.18,30.576376
244,AAPL240719C00160000,2024-07-19,160.0,31.65,30.965406
291,AAPL240816C00160000,2024-08-16,160.0,32.43,31.482635
353,AAPL240920C00160000,2024-09-20,160.0,33.43,32.126804
399,AAPL241018C00160000,2024-10-18,160.0,34.80,32.640253
